<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkJsonFlatten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkJsonFlatten').getOrCreate()

In [4]:
data=[{
		"id": "0001",
		"type": "donut",
		"name": "Cake",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" },
						{ "id": "1003", "type": "Blueberry" },
						{ "id": "1004", "type": "Devil's Food" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5007", "type": "Powdered Sugar" },
				{ "id": "5006", "type": "Chocolate with Sprinkles" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0002",
		"type": "donut",
		"name": "Raised",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0003",
		"type": "donut",
		"name": "Old Fashioned",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	}]

In [5]:
import json
json_data = [json.dumps(record) for record in data]
rdd = spark.sparkContext.parallelize(json_data)


In [6]:
df = spark.read.json(rdd)
df.show()

+--------------------+----+-------------+----+--------------------+-----+
|             batters|  id|         name| ppu|             topping| type|
+--------------------+----+-------------+----+--------------------+-----+
|{[{1001, Regular}...|0001|         Cake|0.55|[{5001, None}, {5...|donut|
| {[{1001, Regular}]}|0002|       Raised|0.55|[{5001, None}, {5...|donut|
|{[{1001, Regular}...|0003|Old Fashioned|0.55|[{5001, None}, {5...|donut|
+--------------------+----+-------------+----+--------------------+-----+



In [7]:
from pyspark.sql.functions import explode, col

# Explode batters
df_exploded = df.withColumn("batter", explode(col("batters.batter"))) \
                .withColumn("topping", explode(col("topping")))

# Select and flatten fields
df_flat = df_exploded.select(
    "id", "type", "name", "ppu",
    col("batter.id").alias("batter_id"),
    col("batter.type").alias("batter_type"),
    col("topping.id").alias("topping_id"),
    col("topping.type").alias("topping_type")
)

df_flat.show(truncate=False)


+----+-----+----+----+---------+-----------+----------+------------------------+
|id  |type |name|ppu |batter_id|batter_type|topping_id|topping_type            |
+----+-----+----+----+---------+-----------+----------+------------------------+
|0001|donut|Cake|0.55|1001     |Regular    |5001      |None                    |
|0001|donut|Cake|0.55|1001     |Regular    |5002      |Glazed                  |
|0001|donut|Cake|0.55|1001     |Regular    |5005      |Sugar                   |
|0001|donut|Cake|0.55|1001     |Regular    |5007      |Powdered Sugar          |
|0001|donut|Cake|0.55|1001     |Regular    |5006      |Chocolate with Sprinkles|
|0001|donut|Cake|0.55|1001     |Regular    |5003      |Chocolate               |
|0001|donut|Cake|0.55|1001     |Regular    |5004      |Maple                   |
|0001|donut|Cake|0.55|1002     |Chocolate  |5001      |None                    |
|0001|donut|Cake|0.55|1002     |Chocolate  |5002      |Glazed                  |
|0001|donut|Cake|0.55|1002  

In [8]:
df2=spark.read.option("multiline","true").json("/content/drive/MyDrive/Abc/postgreSQL/total-alcohol-consumption-per-capita-litres-of-pure-alcohol.metadata.json")
df2.show(  )

+--------------------+--------------------+--------------+
|               chart|             columns|dateDownloaded|
+--------------------+--------------------+--------------+
|{World Health Org...|{{World Health Or...|    2025-03-12|
+--------------------+--------------------+--------------+

